## CIS 335 Assignment 2

Jensen Holm <br>
Feb. 26th 2023

In [9]:
import pandas as pd
from dataclasses import dataclass
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
)

In [4]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

train = train.drop("Unnamed: 32", axis=1)
test = test.drop("Unnamed: 32", axis=1)

train["diagnosis"] = train["diagnosis"].dropna()
test["diagnosis"]= train["diagnosis"].dropna()

X_train = train[[col for col in train.columns if col != "diagnosis"]]
Y_train = train["diagnosis"]

X_test = test[[col for col in train.columns if col != "diagnosis"]]
Y_test = test["diagnosis"]

train.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

## Model Expiramentation

In [10]:

@dataclass
class ClassifierTree:
    X_train: pd.DataFrame
    Y_train: pd.DataFrame
    X_test: pd.DataFrame
    Y_test: pd.DataFrame

    @property
    def x_train(self) -> pd.DataFrame:
        return self.X_train

    @property
    def y_train(self) -> pd.DataFrame:
        return self.Y_train
    
    @property
    def x_test(self) -> pd.DataFrame:
        return self.X_test
    
    @property
    def y_test(self) -> pd.DataFrame:
        return self.Y_test

    def fit(self, norm_type=None, depth=5):
        dt = DecisionTreeClassifier(max_depth=depth)
        dt.fit(self.x_train, self.y_train)
        pred = dt.predict(self.x_test)
        if not norm_type:
            return accuracy_score(
                y_pred=pred,
                y_true=self.y_test,
            )


In [11]:
tree = ClassifierTree(
    X_train=X_train,
    Y_train=Y_train,
    X_test=X_test,
    Y_test=Y_test,
)

#### No Preprocessing

0.39166666666666666

In [ ]:
no_preprocess_dt2 = DecisionTreeClassifier(
    max_depth=6
)

no_preprocess_dt2.fit(X_train, Y_train)

Y_pred2 = no_preprocess_dt2.predict(X_test)

accuracy_score(
    y_true=
)

#### Z-Score Normalization

#### MinMax Normalization